In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from scipy import io

mat = io.loadmat(path+"/D.mat")['X']
print("D")
print(mat.shape)

mat = io.loadmat(path+"/T.mat")['X']
print("T")
print(mat.shape)

mat = io.loadmat(path+"/R.mat")['X']
print("R")
print(mat.shape)

### **Data Compilation**

In [0]:
import sys
import os
from scipy.sparse import dok_matrix, csr_matrix
from scipy import io
import tarfile



#################################################
total_no_users = 2649429
total_no_movies = 17770

def process_content(content, D):
    lines = content.split("\n")
    id_movie = int(lines[0][:-1]) - 1
    for i in range(1, len(lines)):
        if lines[i] != '':
            line = lines[i].split(",")
            id_user = int(line[0]) - 1
            rating = int(line[1])
            D[id_user, id_movie] = rating
    return D


def rating_compiler(folder_name, out_path):
    D = dok_matrix((total_no_users, total_no_movies))
    res_listdir = os.listdir(folder_name)
    number = len(res_listdir)
    i = 0
    for f in res_listdir:
        if os.path.isfile(folder_name+f):
            print(i, " / ", number)
            myfile = open(folder_name+f)
            content = myfile.read()
            myfile.close()
            D = process_content(content, D)
        i += 1
    D = csr_matrix(D)             
    io.savemat(out_path, {'X' : D})


def rating_compiler2(tar_name, out_path):
    D = dok_matrix((total_no_users, total_no_movies))
    tar = tarfile.open(tar_name)
    res_getmembers = tar.getmembers()
    number = len(res_getmembers)
    i = 0
    for member in res_getmembers:
        f = tar.extractfile(member)
        if f is not None:    
            print(i, " / ", number)        
            content = f.read()
            f.close()
            D = process_content(content.decode(), D)
        i += 1
    tar.close()
    D = csr_matrix(D)             
    io.savemat(out_path, {'X' : D})


def extract_T_and_R(D_file_name, file_name, out_T_path, out_R_path):
    D = io.loadmat(D_file_name)['X']
    myfile = open(file_name)
    content = myfile.read()
    myfile.close()
    lines = content.split("\n")
    users, movies = set(), set()
    for line in lines:
        if line != '':
            line_split = line.split(":")
            if len(line_split) == 2:
                # Movie id
                movies.add(int(line_split[0]) - 1)
            else:
                # User id
                users.add(int(line_split[0]) - 1)
    T = D[list(users),:]
    T = T[:,list(movies)]    
    io.savemat(out_T_path, {'X' : T})
    
    movies2 = set(range(total_no_movies))
    movies2 = movies2.difference(movies)
    users2 = set(range(total_no_users))
    users2 = users2.difference(users)
    
    R = D[list(users2),:]
    R = R[:,list(movies2)]
    io.savemat(out_R_path, {'X' : R})
    



#################################################
if __name__ == "__main__":
    #rating_compiler2(path+"/download/training_set.tar", path+"/D.mat")
    #extract_T_and_R(path+"/D.mat", path+"/download/qualifying.txt", path+"/T.mat", path+"/R.mat")
    extract_T_and_R(path+"/D.mat", path+"/download/probe.txt", path+"/T.mat", path+"/R.mat")

### **Utils**

In [0]:
from itertools import groupby
from operator import itemgetter
import pickle
import os
from sklearn.preprocessing import StandardScaler

path = "/content/drive/My Drive/M2/AFMatriciel"

def compute_sparse_correlation_matrix(A):
    scaler = StandardScaler(with_mean=False)
    scaled_A = scaler.fit_transform(A)  # Assuming A is a CSR or CSC matrix
    corr_matrix = (1/scaled_A.shape[0]) * (scaled_A.T @ scaled_A)
    return corr_matrix

def pre_processing(mat, mat_file):
    # Create bu and bi indexes
    # bi_index is a list with a size equal to the number of users
    #    the jth element is a list storing the indexes of movies rated by user j
    # bu_index is the same but storing the indexes of users whose rating is 
    #    available for a movie
    # These indexes will help to vectorize computation of the gradient

    shape = str(mat.shape[0])+"_"+str(mat.shape[1])
    bu_index_file = mat_file+"_bu_index_"+shape+".data"
    bi_index_file = mat_file+"_bi_index_"+shape+".data"

    if not (os.path.isfile(bu_index_file) and os.path.isfile(bi_index_file)):
        #mat = io.loadmat(mat_file)['X']
        """mat = mat[1:5000,1:5000]
        mat = mat[mat.getnnz(1)>0][:,mat.getnnz(0)>0]"""

        print("Pre-processing...")
        mat_nonzero = mat.nonzero()
        """cx = mat.tocoo()    
        bi_index = [[]]*mat.shape[0]
        bu_index = [[]]*mat.shape[1]
        for i,j,v in zip(cx.row, cx.col, cx.data):
          bi_index[i].append(j)
          bu_index[j].append(i)
        print(bi_index[0])"""

        print("   make bi indexes...")
        bi_index = []
        for k, g in groupby(zip(mat_nonzero[0], mat_nonzero[1]), itemgetter(0)):
          to_add = list(map(lambda x:int(x[1]), list(g)))
          bi_index.append(to_add)

        print("   make bu indexes...")
        bu_index = []
        indexes = np.argsort(mat_nonzero[1])
        for k, g in groupby(zip(mat_nonzero[1][indexes], mat_nonzero[0][indexes]), itemgetter(0)):
          to_add = list(map(lambda x:int(x[1]), list(g)))
          bu_index.append(to_add)    

        with open(bi_index_file, "wb") as fp:
            pickle.dump(bi_index, fp)
        with open(bu_index_file, "wb") as fp:
            pickle.dump(bu_index, fp)
    else:
        with open(bi_index_file, "rb") as fp:
            bi_index = pickle.load(fp)
        with open(bu_index_file, "rb") as fp:
            bu_index = pickle.load(fp)

    print("Pre-processing done.")
    return bu_index, bi_index

### **1. Baseline Estimates**

In [0]:
from scipy import io, sparse
import numpy as np



#################################################
def compute_loss(mat, mu, bu, bi, l_reg=0.02):
  loss = 0

  no_users_entries = np.array((mat != 0).sum(1)).T.ravel()
  bu_rep = np.repeat(bu.ravel(), no_users_entries)

  no_movies_entries = np.array((mat != 0).sum(0)).ravel()
  bi_rep = np.repeat(bi.ravel(), no_movies_entries)

  temp_mat = sparse.csc_matrix(mat).copy()
  temp_mat.data[:] -= bi_rep
  temp_mat.data[:] -= mu
  temp_mat = sparse.coo_matrix(temp_mat)
  temp_mat = sparse.csr_matrix(temp_mat)
  temp_mat.data[:] -= bu_rep

  loss = (temp_mat.data[:] ** 2).sum()

  reg = l_reg * ((bu**2).sum() + (bi**2).sum())  
  loss += reg

  return loss


def baseline_estimator(mat, mat_file, l_reg=0.02, learning_rate=0.0000025):
  # subsample the matrix to make computation faster
  """mat = mat[0:mat.shape[0]//128, 0:mat.shape[1]//128]
  mat = mat[mat.getnnz(1)>0][:, mat.getnnz(0)>0]"""

  print(mat.shape)
  no_users = mat.shape[0]
  no_movies = mat.shape[1]
  
  bu_index, bi_index = pre_processing(mat, mat_file)

  bu = np.random.rand(no_users,1)  * 2 - 1
  bi = np.random.rand(1,no_movies) * 2 - 1
  #bu = np.zeros((no_users,1))
  #bi = np.zeros((1,no_movies))  

  mu = mat.data[:].mean()
  mat_sum1 = mat.sum(1)
  mat_sum0 = mat.sum(0)
  n = mat.data[:].shape[0]

  no_users_entries = np.array((mat != 0).sum(1))
  no_movies_entries = np.array((mat != 0).sum(0))

  # Train
  print("Train...")
  n_iter = 200
  for it in range(n_iter):

    #bi_sum = bi[bi_index].sum(1).reshape((no_users,1))
    #bu_sum = bu.ravel()[bu_index].sum(0).reshape((1,no_movies)) 

    bi_sum = np.array(list(map(lambda x:bi.ravel()[x].sum(), bi_index))).reshape((no_users,1))
    bu_sum = np.array(list(map(lambda x:bu.ravel()[x].sum(), bu_index))).reshape((1,no_movies))    

    # Vectorized operations
    bu_gradient = - 2.0 * (mat_sum1 - no_users_entries  * mu - no_users_entries  * bu - bi_sum) + 2.0 * l_reg * bu
    bu -= learning_rate * bu_gradient 

    bi_gradient = - 2.0 * (mat_sum0 - no_movies_entries * mu - no_movies_entries * bi - bu_sum) + 2.0 * l_reg * bi
    bi -= learning_rate * bi_gradient 
 
    if it % 10 == 0:
      print("compute loss...")
      print(compute_loss(mat, mu, bu, bi, l_reg=l_reg))

  return bu, bi
#################################################



if __name__ == "__main__":
    mat_file = path+"/T.mat"
    mat = io.loadmat(mat_file)['X']    
    baseline_estimator(mat, mat_file)

### **2. Correlation-Based Neighbourhood Model**

In [0]:
import numpy as np
from scipy import io, sparse
from math import sqrt
from time import time



# Through all this code Rk_iu and Nk_iu are the same since implicit matrix is
#    made from the rating matrix without additional information (i.e. indexes of
#    non-zero elements are the same therefore neighbors too).


#################################################
# Non-vectorized way (iterate through each r_ui)
#################################################
def predict_r_ui(mat, u, i, mu, S, Sk_iu, baseline_bu, baseline_bi):
  bui = mu + baseline_bu[u] + baseline_bi[0, i]
  buj = mu + baseline_bu[u] + baseline_bi[0, Sk_iu]
  return bui + 1 / S[i, Sk_iu].sum() * (S[i, Sk_iu].toarray().ravel() * (mat[u, Sk_iu].toarray().ravel() - buj)).sum()

def correlation_based_neighbourhood_model(mat, mat_file, l_reg2=100.0, k=250):
    # subsample the matrix to make computation faster
    """mat = mat[0:mat.shape[0]//128, 0:mat.shape[1]//128]
    mat = mat[mat.getnnz(1)>0][:, mat.getnnz(0)>0]"""

    print(mat.shape)
    no_users = mat.shape[0]
    no_movies = mat.shape[1]

    #baseline_bu, baseline_bi = baseline_estimator(mat)
    # We should call baseline_estimator but we can init at random for test
    baseline_bu, baseline_bi = np.random.rand(no_users, 1)  * 2 - 1, np.random.rand(1, no_movies) * 2 - 1    

    #bu_index, bi_index = pre_processing(mat, mat_file)

    mu = mat.data[:].mean()

    # Compute similarity matrix
    N = sparse.csr_matrix(mat).copy()
    N.data[:] = 1
    S = sparse.csr_matrix.dot(N.T, N)
    S.data[:] = S.data[:] / (S.data[:] + l_reg2)
    S = S * compute_sparse_correlation_matrix(mat)

    # Computation
    print("Computation...")
    n_iter = 200
    cx = mat.tocoo()
    r_ui_mat = []
    for u,i,v in zip(cx.row, cx.col, cx.data):
        Sk_iu = np.flip(np.argsort(S[i,].toarray()))[:k].ravel()
        r_ui = predict_r_ui(mat, u, i, mu, S, Sk_iu, baseline_bu, baseline_bi)
        r_ui_mat.append((u, i, r_ui))

    return r_ui_mat

#################################################


if __name__ == "__main__":
    mat_file = path+"/T.mat"
    mat = io.loadmat(mat_file)['X']
    correlation_based_neighbourhood_model(mat, mat_file)

### **3. Correlation-Based Neighbourhood Model with Implicit Feedback**

In [0]:
import numpy as np
from scipy import io, sparse
from math import sqrt
from time import time



# Through all this code Rk_iu and Nk_iu are the same since implicit matrix is
#    made from the rating matrix without additional information (i.e. indexes of
#    non-zero elements are the same therefore neighbors too).


#################################################
# Non-vectorized way (iterate through each r_ui)
#################################################
def predict_r_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi):
    buj = mu + baseline_bu[u] + baseline_bi[0, Rk_iu]
    Rk_iu_sum = np.multiply((mat[u, Rk_iu] - buj), wij[i][Rk_iu]).sum()
    Nk_iu_sum = cij[i][Rk_iu].sum()
    return mu + bu[u] + bi[0, i] + Rk_iu_sum / sqrt(len(Rk_iu)) + Nk_iu_sum / sqrt(len(Nk_iu))

def compute_e_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi):
    return mat[u, i] - predict_r_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi)

def compute_loss(mat, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, l_reg=0.002):
    loss = 0
    cx = mat.tocoo()        
    for u,i,v in zip(cx.row, cx.col, cx.data):
        r_ui_pred = predict_r_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi)
        Rk_iu_sum = (wij[i][Rk_iu] ** 2).sum()
        Nk_iu_sum = (cij[i][Rk_iu] ** 2).sum()
        loss += (mat[u, i] - r_ui_pred) ** 2 + l_reg * ((bu ** 2).sum() + (bi ** 2).sum() + Rk_iu_sum + Nk_iu_sum) 

    return loss

def correlation_based_implicit_neighbourhood_model(mat, mat_file, l_reg=0.002, gamma=0.005, l_reg2=100.0, k=250):
    # subsample the matrix to make computation faster
    """mat = mat[0:mat.shape[0]//128, 0:mat.shape[1]//128]
    mat = mat[mat.getnnz(1)>0][:, mat.getnnz(0)>0]"""

    print(mat.shape)
    no_users = mat.shape[0]
    no_movies = mat.shape[1]

    #baseline_bu, baseline_bi = baseline_estimator(mat)
    # We should call baseline_estimator but we can init at random for test
    baseline_bu, baseline_bi = np.random.rand(no_users, 1)  * 2 - 1, np.random.rand(1, no_movies) * 2 - 1    

    bu_index, bi_index = pre_processing(mat, mat_file)
    
    # Init parameters
    bu = np.random.rand(no_users, 1)  * 2 - 1
    bi = np.random.rand(1, no_movies) * 2 - 1
    wij = np.random.rand(no_movies, no_movies) * 2 - 1
    cij = np.random.rand(no_movies, no_movies) * 2 - 1

    mu = mat.data[:].mean()

    # Compute similarity matrix
    N = sparse.csr_matrix(mat).copy()
    N.data[:] = 1
    S = sparse.csr_matrix.dot(N.T, N)
    S.data[:] = S.data[:] / (S.data[:] + l_reg2)
    S = S * compute_sparse_correlation_matrix(mat)

    # Train
    print("Train...")
    n_iter = 200
    cx = mat.tocoo()        
    for it in range(n_iter):
        t0 = time()
        for u,i,v in zip(cx.row, cx.col, cx.data):
            #Rk_iu = Nk_iu = bi_index[u]
            Rk_iu = Nk_iu = np.flip(np.argsort(S[i,].toarray()))[:k].ravel()
            e_ui = compute_e_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi)

            bu[u] += gamma * (e_ui - l_reg * bu[u])
            bi[0, i] += gamma * (e_ui - l_reg * bi[0, i])

            buj = mu + baseline_bu[u] + baseline_bi[0, Rk_iu]
            wij[i][Rk_iu] += gamma * ( 1 / sqrt(len(Rk_iu)) * e_ui * (mat[u, Rk_iu].toarray().ravel() - buj) - l_reg * wij[i][Rk_iu] )
            cij[i][Nk_iu] += gamma * ( 1 / sqrt(len(Nk_iu)) * e_ui - l_reg * cij[i][Nk_iu] )
        gamma *= 0.99

        if it % 10 == 0:
          t1 = time()
          print(it, "\ ", n_iter, "(%.2g sec)" % (t1 - t0))
          print("compute loss...")
          print(compute_loss(mat, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, l_reg=l_reg))

    return bu, bi, wij, cij
#################################################


#################################################
# Vectorized way (in work)
# (Actually this version is faster but updates e_ui
# less frequently making it less accurate for the
# gradient descent)
#################################################
def compute_e_vectorized(mat, mu, bu, bi, Rk, wij, Nk, cij, baseline_bu, baseline_bi):
    # Rk and Nk are list of tuple (u, i, Rk_iu/Nk_iu)

    no_users_entries = np.array((mat != 0).sum(1)).T.ravel()
    bu_rep = np.repeat(bu.ravel(), no_users_entries)

    no_movies_entries = np.array((mat != 0).sum(0)).ravel()
    bi_rep = np.repeat(bi.ravel(), no_movies_entries)

    temp_mat = sparse.csc_matrix(mat).copy()
    temp_mat.data[:] -= mu
    temp_mat.data[:] -= bi_rep
    Rk_sum = np.array(list(map(lambda x : ( (mat[x[0], x[2]].toarray().ravel() \
                                           - (mu + baseline_bu[x[0]] + baseline_bi[0, x[2]])) \
                                           * wij[x[1]][x[2]] ).sum() / sqrt(len(x[2])), Rk)))
    temp_mat.data[:] -= Rk_sum
    Nk_sum = np.array(list(map(lambda x : cij[x[1]][x[2]].sum() / sqrt(len(x[2])), Nk)))
    temp_mat.data[:] -= Nk_sum
    temp_mat = sparse.coo_matrix(temp_mat)
    temp_mat = sparse.csr_matrix(temp_mat)
    temp_mat.data[:] -= bu_rep

    return temp_mat

def compute_loss_vectorized(mat, mu, bu, bi, Rk, wij, Nk, cij, baseline_bu, baseline_bi, l_reg=0.002):
    no_nonzero_element = np.array((mat != 0).sum())
    loss = (compute_e_vectorized(mat, mu, bu, bi, Rk, wij, Nk, cij, baseline_bu, baseline_bi).data[:] ** 2).sum()
    loss += l_reg * np.array(list(map(lambda x : (cij[x[1]][x[2]] ** 2).sum(), Nk))).sum()
    loss += l_reg * np.array(list(map(lambda x : (wij[x[1]][x[2]] ** 2).sum(), Rk))).sum()
    loss += no_nonzero_element * l_reg * (bu ** 2).sum()
    loss += no_nonzero_element * l_reg * (bi ** 2).sum()

    return loss

def correlation_based_implicit_neighbourhood_model_vectorized(mat, mat_file, l_reg=0.002, gamma=0.005, l_reg2=100.0, k=250):
    gamma /= 100

    # subsample the matrix to make computation faster
    mat = mat[0:mat.shape[0]//128, 0:mat.shape[1]//128]
    mat = mat[mat.getnnz(1)>0][:, mat.getnnz(0)>0]

    print(mat.shape)
    no_users = mat.shape[0]
    no_movies = mat.shape[1]
    no_users_entries = np.array((mat != 0).sum(1))
    no_movies_entries = np.array((mat != 0).sum(0))    

    #baseline_bu, baseline_bi = baseline_estimator(mat)
    # We should call baseline_estimator but we can init at random for testing
    baseline_bu, baseline_bi = np.random.rand(no_users, 1)  * 2 - 1, np.random.rand(1, no_movies) * 2 - 1    

    bu_index, bi_index = pre_processing(mat, mat_file)
    
    bu = np.random.rand(no_users, 1)  * 2 - 1
    bi = np.random.rand(1, no_movies) * 2 - 1
    wij = np.random.rand(no_movies, no_movies) * 2 - 1
    cij = np.random.rand(no_movies, no_movies) * 2 - 1

    mu = mat.data[:].mean()

    # Compute similarity matrix
    N = sparse.csr_matrix(mat).copy()
    N.data[:] = 1
    S = sparse.csr_matrix.dot(N.T, N)
    S.data[:] = S.data[:] / (S.data[:] + l_reg2)
    S = S * compute_sparse_correlation_matrix(mat)

    Rk = []
    cx = mat.tocoo()
    for u,i,v in zip(cx.row, cx.col, cx.data):
        Rk.append((u, i, np.flip(np.argsort(S[i,].toarray()))[:k].ravel()))

    # Train
    print("Train...")
    n_iter = 200
    for it in range(n_iter):
        t0 = time() 

        e = compute_e_vectorized(mat, mu, bu, bi, Rk, wij, Rk, cij, baseline_bu, baseline_bi)
        # Vectorized operations
        bu += gamma * (e.sum(1) - no_users_entries * l_reg * bu)
        bi += gamma * (e.sum(0) - no_movies_entries * l_reg * bi)

        # TODO: vectorize the following
        for u, i, Rk_iu in Rk:
            Nk_iu = Rk_iu
            e_ui = e[u, i]
            buj = mu + baseline_bu[u] + baseline_bi[0, Rk_iu]
            wij[i][Rk_iu] += gamma * ( 1 / sqrt(len(Rk_iu)) * e_ui * (mat[u, Rk_iu].toarray().ravel() - buj) - l_reg * wij[i][Rk_iu] )
            cij[i][Nk_iu] += gamma * ( 1 / sqrt(len(Nk_iu)) * e_ui - l_reg * cij[i][Nk_iu] )
        gamma *= 0.99

        if it % 10 == 0:
          t1 = time()
          print(it, "\ ", n_iter, "(%.2g sec)" % (t1 - t0))       
          print("compute loss...")
          print(compute_loss_vectorized(mat, mu, bu, bi, Rk, wij, Rk, cij, baseline_bu, baseline_bi, l_reg=l_reg))  

    return bu, bi, wij, cij
#################################################


if __name__ == "__main__":
    mat_file = path+"/T.mat"
    mat = io.loadmat(mat_file)['X']
    correlation_based_implicit_neighbourhood_model(mat, mat_file)
    #correlation_based_implicit_neighbourhood_model_vectorized(mat, mat_file)

In [0]:
from scipy import io
import numpy as np
mat = io.loadmat(path+"/T.mat")['X']

no_users = mat.shape[0]
no_movies = mat.shape[1]
bu = np.random.rand(no_users, 1)  * 2 - 1
bi = np.random.rand(1, no_movies) * 2 - 1

print(bi.shape)
print(bi[0].shape)
print(bi[0, 0].shape)

"""a = np.corrcoef(mat)
print(a)"""

### **4. SVD++**

In [0]:
import numpy as np
from scipy import io, sparse
from math import sqrt, isnan



#################################################
# Non-vectorized way
#################################################
def predict_r_ui(mat, u, i, mu, bu, bi, qi, pu, N_u, yj):
    N_u_sum = yj[N_u].sum(0)
    return mu + bu[u] + bi[0, i] + np.dot(qi[i], (pu[u] + N_u_sum / sqrt(len(N_u))))

def compute_e_ui(mat, u, i, mu, bu, bi, qi, pu, N_u, yj):
    return mat[u, i] - predict_r_ui(mat, u, i, mu, bu, bi, qi, pu, N_u, yj)

def compute_loss(mat, mu, bu, bi, qi, pu, N_u, yj, l_reg6=0.005, l_reg7=0.015):
    loss = 0
    cx = mat.tocoo()
    for u,i,v in zip(cx.row, cx.col, cx.data):
        r_ui_pred = predict_r_ui(mat, u, i, mu, bu, bi, qi, pu, N_u, yj)
        loss += (mat[u, i] - r_ui_pred) ** 2 + l_reg6 * ((bu ** 2).sum() + (bi ** 2).sum())
        loss += l_reg7 * ((qi[i]**2).sum() + (pu[u]**2).sum() + (yj[N_u]**2).sum())

    return loss

def svd_more_more(mat, mat_file, gamma1=0.007, gamma2=0.007, gamma3=0.001, l_reg2=100, l_reg6=0.005, l_reg7=0.015, f=50):
    # subsample the matrix to make computation faster
    """mat = mat[0:mat.shape[0]//128, 0:mat.shape[1]//128]
    mat = mat[mat.getnnz(1)>0][:, mat.getnnz(0)>0]"""

    print(mat.shape)
    no_users = mat.shape[0]
    no_movies = mat.shape[1]

    bu_index, bi_index = pre_processing(mat, mat_file)
    
    # Init parameters
    bu = np.random.rand(no_users, 1)  * 2 - 1
    bi = np.random.rand(1, no_movies) * 2 - 1
    qi = np.random.rand(no_movies, f) * 2 - 1
    pu = np.random.rand(no_users, f) * 2 - 1
    yj = np.random.rand(no_movies, f) * 2 - 1

    mu = mat.data[:].mean()

    # Train
    print("Train...")
    n_iter = 200
    cx = mat.tocoo()
    for it in range(n_iter):
        for u,i,v in zip(cx.row, cx.col, cx.data):
            N_u = bi_index[u]
            e_ui = compute_e_ui(mat, u, i, mu, bu, bi, qi, pu, N_u, yj)

            bu[u] += gamma1 * (e_ui - l_reg6 * bu[u])
            bi[0, i] += gamma1 * (e_ui - l_reg6 * bi[0, i])
            qi[i] += gamma2 * (e_ui * (pu[u] + 1 / sqrt(len(N_u)) * yj[N_u].sum(0)) - l_reg7 * qi[i])
            pu[u] += gamma2 * (e_ui * qi[i] - l_reg7 * pu[u])
            yj[N_u] += gamma2 * (e_ui * 1/ sqrt(len(N_u)) * qi[i] - l_reg7 * yj[N_u])
        gamma1 *= 0.9
        gamma2 *= 0.9

        if it % 10 == 0:
          print(it, "\ ", n_iter)         
          print("compute loss...")
          print(compute_loss(mat, mu, bu, bi, qi, pu, N_u, yj, l_reg6=l_reg6, l_reg7=l_reg7))
    
    return bu, bi, qi, pu, yj
#################################################


if __name__ == "__main__":
    mat_file = path+"/T.mat"
    mat = io.loadmat(mat_file)['X']
    svd_more_more(mat, mat_file)

### **5. Integrated Model**

In [0]:
import numpy as np
from scipy import io, sparse
from math import sqrt, isnan



# Through all this code Rk_iu and Nk_iu are the same since implicit matrix is
#    made from the rating matrix without additional information.


#################################################
# Non-vectorized way
#################################################
def predict_r_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, qi, pu, N_u, yj):
    buj = mu + baseline_bu[u] + baseline_bi[0, Rk_iu]
    Rk_iu_sum = np.multiply((mat[u, Rk_iu] - buj), wij[i][Rk_iu]).sum()
    Nk_iu_sum = cij[i][Rk_iu].sum()
    N_u_sum = yj[N_u].sum(0)
    return mu + bu[u] + bi[0, i] + np.dot(qi[i], (pu[u] + N_u_sum / sqrt(len(N_u)))) + Rk_iu_sum / sqrt(len(Rk_iu)) + Nk_iu_sum / sqrt(len(Nk_iu))

def compute_e_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, qi, pu, N_u, yj):
    return mat[u, i] - predict_r_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, qi, pu, N_u, yj)

def compute_loss(mat, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, qi, pu, N_u, yj, l_reg6=0.005, l_reg7=0.015, l_reg8=0.015):
    loss = 0
    cx = mat.tocoo()
    for u,i,v in zip(cx.row, cx.col, cx.data):
        r_ui_pred = predict_r_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, qi, pu, N_u, yj)
        Rk_iu_sum = (wij[i][Rk_iu] ** 2).sum()
        Nk_iu_sum = (cij[i][Rk_iu] ** 2).sum()
        loss += (mat[u, i] - r_ui_pred) ** 2 + l_reg6 * ((bu ** 2).sum() + (bi ** 2).sum()) + l_reg8 * (Rk_iu_sum + Nk_iu_sum)
        loss += l_reg7 * ((qi[i]**2).sum() + (pu[u]**2).sum() + (yj[N_u]**2).sum())

    return loss

def integrated_model(mat, mat_file, gamma1=0.007, gamma2=0.007, gamma3=0.001, l_reg2=100, l_reg6=0.005, l_reg7=0.015, l_reg8=0.015, k=300, f=50):
    # subsample the matrix to make computation faster
    """mat = mat[0:mat.shape[0]//128, 0:mat.shape[1]//128]
    mat = mat[mat.getnnz(1)>0][:, mat.getnnz(0)>0]"""

    print(mat.shape)
    no_users = mat.shape[0]
    no_movies = mat.shape[1]

    #baseline_bu, baseline_bi = baseline_estimator(mat)
    # We should call baseline_estimator but we can init at random for test
    baseline_bu, baseline_bi = np.random.rand(no_users, 1)  * 2 - 1, np.random.rand(1, no_movies) * 2 - 1    

    bu_index, bi_index = pre_processing(mat, mat_file)
    
    # Init parameters
    bu = np.random.rand(no_users, 1)  * 2 - 1
    bi = np.random.rand(1, no_movies) * 2 - 1
    wij = np.random.rand(no_movies, no_movies) * 2 - 1
    cij = np.random.rand(no_movies, no_movies) * 2 - 1
    qi = np.random.rand(no_movies, f) * 2 - 1
    pu = np.random.rand(no_users, f) * 2 - 1
    yj = np.random.rand(no_movies, f) * 2 - 1

    mu = mat.data[:].mean()
    N = sparse.csr_matrix(mat).copy()
    N.data[:] = 1
    S = sparse.csr_matrix.dot(N.T, N)
    S.data[:] = S.data[:] / (S.data[:] + l_reg2)
    S = S * compute_sparse_correlation_matrix(mat)

    # Train
    print("Train...")
    n_iter = 200
    cx = mat.tocoo()
    for it in range(n_iter):
        for u,i,v in zip(cx.row, cx.col, cx.data):
            #Rk_iu = Nk_iu = bi_index[u]
            N_u = bi_index[u]
            Rk_iu = Nk_iu = np.flip(np.argsort(S[i,].toarray()))[:k].ravel()
            e_ui = compute_e_ui(mat, u, i, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, qi, pu, N_u, yj)

            bu[u] += gamma1 * (e_ui - l_reg6 * bu[u])
            bi[0, i] += gamma1 * (e_ui - l_reg6 * bi[0, i])
            qi[i] += gamma2 * (e_ui * (pu[u] + 1 / sqrt(len(N_u)) * yj[N_u].sum(0)) - l_reg7 * qi[i])
            pu[u] += gamma2 * (e_ui * qi[i] - l_reg7 * pu[u])
            yj[N_u] += gamma2 * (e_ui * 1/ sqrt(len(N_u)) * qi[i] - l_reg7 * yj[N_u])
            buj = mu + baseline_bu[u] + baseline_bi[0, Rk_iu]
            wij[i][Rk_iu] += gamma3 * ( 1 / sqrt(len(Rk_iu)) * e_ui * (mat[u, Rk_iu].toarray().ravel() - buj) - l_reg8 * wij[i][Rk_iu] )
            cij[i][Nk_iu] += gamma3 * ( 1 / sqrt(len(Nk_iu)) * e_ui - l_reg8 * cij[i][Nk_iu] )                
        gamma1 *= 0.9
        gamma2 *= 0.9
        gamma3 *= 0.9

        if it % 10 == 0:
          print(it, "\ ", n_iter)         
          print("compute loss...")
          print(compute_loss(mat, mu, bu, bi, Rk_iu, wij, Nk_iu, cij, baseline_bu, baseline_bi, qi, pu, N_u, yj, l_reg6=l_reg6, l_reg7=l_reg7, l_reg8=l_reg8))

    return bu, bi, qi, pu, yj, wij, cij
#################################################


if __name__ == "__main__":
    mat_file = path+"/T.mat"
    mat = io.loadmat(mat_file)['X']
    integrated_model(mat, mat_file)